# TO DO:
- How to handle NaN in encoder? (drop before classification)
- Mess with classifier parameters
- test classifier
- how to handle repeats?
- change order of operations so that slave_indexes match unclassified_data...how to do this?


In [2]:
import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import collections

from exploringShipLogbooks.basic_utils import isloate_training_data
from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import clean_data

from exploringShipLogbooks.config import *


# Load and clean data
### Load CLIWOC ship logs

In [7]:
# extract data from zip file
cliwoc_data = extract_logbook_data('CLIWOC15.csv')
cliwoc_data = cliwoc_data.loc[50000:100000]

C:\Users\Emma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Find definite slave data in CLIWOC data set
- These logs will be used to test the classifier

In [8]:
# extract logs that mention slaves
mentions_slaves = wc.count_key_words(cliwoc_data, text_columns, slave_words)
slave_index = mentions_slaves[(mentions_slaves['ContainsKeyWord'] != 0)].index

print('Found ', len(slave_index), ' logs that mention slaves')

# cleanup
del mentions_slaves

Found  10  logs that mention slaves


### Clean CLIWOC data

In [9]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

# clean data (make all same case)
cliwoc_data = clean_data(cliwoc_data)

### Load Slave Voyages data

In [38]:
file_name = './exploringShipLogbooks/data/tastdb-exp-2010'
slave_voyage_logs = df = pd.read_pickle(file_name)

### Clean Slave voyages data

In [11]:
slave_voyage_desired_cols = ['portdep', 'portret', 'shipname', 
                             'rig', 'national', 'yeardep']
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.columns = ['ShipName', 'Nationality', 'ShipType', 
                             'VoyageFrom', 'VoyageTo', 'Year']

slave_voyage_logs = clean_data(slave_voyage_logs)

# Join data sets

In [26]:
all_data = pd.concat([cliwoc_data, slave_voyage_logs], ignore_index = True)
all_data = clean_data(all_data)

# keep track of slave training data indices
last_cliwoc_index = len(cliwoc_data)
new_last_index = len(slave_voyage_logs) + last_cliwoc_index
training_slave_indices = list(range(last_cliwoc_index, new_last_index))

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [ ]:
all_data = encode_data_df(all_data)

# Extract training data, and create list of classes

In [18]:
# slave_logs training data is from slave voyages data set
slave_logs = all_data.iloc[training_slave_indices]

criteria = {'ShipName': non_slave_ships}
no_slave_logs, no_slave_mask = isloate_training_data(cliwoc_data, criteria)

# remaining data is unclassified 
unclassified_logs = all_data.drop(all_data.index[training_slave_indices])

# convert to numpy array
unclassified_logs = unclassified_logs.as_matrix()

# clean-up
del all_data

['assurance', 'san carlos', 'san perpetua', 'severn', 'la atrevida', 'el cuervo', 'el carlos real', 'greyhound', 'jason', 'la perle', 'la leopard', 'scipio', 'princes louisa', 'pollux']
50001
50001


In [25]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(no_slave_logs))
classes = np.append(classes, np.ones(len(slave_logs)))

# joint training data
training_data = pd.concat([no_slave_logs, slave_logs], ignore_index = True)

# convert to numpy array
training_data = training_data.as_matrix()

# cleanup
del no_slave_logs, slave_logs

# Fit training data to classifier
- **note!** first column of numpy array is index! do not include in classification!

In [11]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data, classes)

predictions = classifier.predict(unclassified_logs)

In [12]:
predictions_count = collections.Counter(predictions)
print(predictions_count)

Counter({1.0: 219108, 0.0: 60172})


# Test classifier
- check if slave logs from cliwoc data were classified correctly (want mostly classified as 1)
- compare first column with slave_index
